In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

# GTS

In [3]:
from models.GTS.gts_graph_learning import GTS_Graph_Learning
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.DCRNN import DCRNN
from utils.utils import build_fully_connected_edge_idx

from glob import glob
import yaml
from easydict import EasyDict as edict

In [4]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [5]:
# node_feas = torch.rand(config.nodes_num, config.node_features, 1000)
# edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [6]:
spike = pickle.load(open('./data/spk_bin_n100.pickle', 'rb'))

spike = torch.FloatTensor(spike[:,:4800])

In [7]:
config.nodes_num = 100

In [8]:
edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [9]:
edge_index.shape

torch.Size([2, 9900])

In [10]:
spike.shape

torch.Size([100, 4800])

In [11]:
gl = GTS_Graph_Learning(config)

In [12]:
gl

GTS_Graph_Learning(
  (conv1): Conv1d(1, 4, kernel_size=(200,), stride=(20,))
  (conv2): Conv1d(4, 4, kernel_size=(80,), stride=(10,))
  (conv3): Conv1d(4, 4, kernel_size=(10,), stride=(5,))
  (fc_conv): Conv1d(4, 1, kernel_size=(1,), stride=(1,))
  (fc): Linear(in_features=2, out_features=16, bias=True)
  (hidden_drop): Dropout(p=0.2, inplace=False)
  (bn1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_cat): Linear(in_features=32, out_features=16, bias=True)
  (fc_out): Linear(in_features=16, out_features=2, bias=True)
)

In [13]:
adj = gl(spike, edge_index)

In [14]:
adj.shape

torch.Size([9900, 2])

In [16]:
adj

tensor([[0.5970, 0.0000],
        [0.5607, 0.0000],
        [0.5829, 0.0000],
        ...,
        [0.6402, 0.0000],
        [0.8975, 0.0000],
        [1.1305, 0.0000]], grad_fn=<ReluBackward0>)

In [ ]:
z_1 = F.gumbel_softmax(adj, tau=0.9, hard=True)

In [ ]:
z_1.shape

In [ ]:
z_1[:,0].shape

In [ ]:
edge_index.shape

In [ ]:
z_adj = torch.where(z_1[:,0])

In [ ]:
z_adj

In [ ]:
b= edge_index[0,:][z_adj]

In [ ]:
c = edge_index[1,:][z_adj]

In [ ]:
a = torch.stack([b,c])

In [ ]:
from torch_geometric.utils import *

In [ ]:
is_undirected(a)

In [ ]:
to_undirected(a)